In [1]:
# analysis tools from scikit-xray (https://github.com/scikit-xray/scikit-xray/tree/master/skxray/core)
import skxray.core.roi as roi
import skxray.core.correlation as corr
import skxray.core.utils as utils

from lmfit import minimize, Parameters, Model

# plotting tools from xray_vision (https://github.com/Nikea/xray-vision/blob/master/xray_vision/mpl_plotting/roi.py)
import xray_vision.mpl_plotting as mpl_plot  

import numpy as np
import os, sys
import zipfile

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import LogNorm

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [ ]:
#data_dir = "Duke_data/"
data_dir = "/Volumes/Data/BeamLines/CHX/Luxi_description_files_for_duke/Duke_data/"
duke_rdata = np.load(data_dir+"duke_img_1_5000.npy")
duke_dark = np.load(data_dir+"duke_dark.npy")

duke_data = []
for i in range(duke_rdata.shape[0]):
    duke_data.append(duke_rdata[i] - duke_dark)
    
duke_ndata=np.asarray(duke_data)

# load the mask(s) and mask the data
mask1 = np.load(data_dir+"new_mask4.npy")
mask2 = np.load(data_dir+"Luxi_duke_mask.npy")

N_mask = ~(mask1 + mask2)
mask_data = N_mask*duke_ndata

In [ ]:
#  get the average image
avg_img = np.average(duke_ndata, axis=0)

# if matplotlib version 1.5 or later
if float('.'.join(mpl.__version__.split('.')[:2])) >= 1.5:
    cmap = 'viridis'
else:
    cmap = 'CMRmap'

# plot the average image data after masking
plt.figure()
plt.imshow(N_mask*avg_img, vmax=1e0, cmap="Paired")
plt.title("Averaged masked data for Duke Silica Gel ")
plt.colorbar()
plt.show()

In [ ]:
center = (133, 143)   # center of the spckle pattern

In [ ]:
inner_radius = 25  # radius of the first ring
width = 1        # width of each ring
spacing = 0      # no spacing between rings
num_rings = 3   # number of rings
center = (133, 143)   # center of the spckle pattern

#  find the edges of the required rings
edges = roi.ring_edges(inner_radius, width, spacing, num_rings)
edges

rings = roi.rings(edges, center, avg_img.shape)

mask_data2 = N_mask*duke_data[0:4999]

ring_mask = rings*N_mask

# plot the figure
fig, axes = plt.subplots()
axes.set_title("Label array on the image")
im = show_label_array_on_image(axes, avg_img*N_mask, ring_mask, cmap="gray", imshow_cmap='Paired')
plt.show()

In [16]:
import copy
import matplotlib.cm as mcm
from matplotlib.colors import LogNorm


def show_label_array(ax, label_array, cmap=None, **kwargs):
    """
    Display a labeled array nicely
    Additional kwargs are passed through to `ax.imshow`.
    If `vmin` is in kwargs, it is clipped to minimum of 0.5.
    Parameters
    ----------
    ax : Axes
        The `Axes` object to add the artist too
    label_array : ndarray
        Expected to be an unsigned integer array.  0 is background,
        positive integers label region of interest
    cmap : str or colormap, optional
        Color map to use, defaults to 'Paired'
    Returns
    -------
    img : AxesImage
        The artist added to the axes
    """
    if cmap is None:
        cmap = 'Paired'

    _cmap = copy.copy((mcm.get_cmap(cmap)))
    _cmap.set_under('w', 0)
    vmin = max(.5, kwargs.pop('vmin', .5))

    ax.set_aspect('equal')
    im = ax.imshow(label_array, cmap=cmap,
                   interpolation='nearest',
                   vmin=vmin,
                   **kwargs)

    return im


def show_label_array_on_image(ax, image, label_array, cmap=None,
                              imshow_cmap='gray', norm=LogNorm(), **kwargs):
    """
    This will plot the required ROI's(labeled array) on the image
    Additional kwargs are passed through to `ax.imshow`.
    If `vmin` is in kwargs, it is clipped to minimum of 0.5.
    Parameters
    ----------
    ax : Axes
        The `Axes` object to add the artist too
    image : array
        The image array
    label_array : array
        Expected to be an unsigned integer array.  0 is background,
        positive integers label region of interest
    cmap : str or colormap, optional
        Color map to use for plotting the label_array, defaults to 'None'
    imshow_cmap : str or colormap, optional
        Color map to use for plotting the image, defaults to 'gray'
    norm : str, optional
        Normalize scale data, defaults to 'Lognorm()'
    Returns
    -------
    im : AxesImage
        The artist added to the axes
    im_label : AxesImage
        The artist added to the axes
    """
    ax.set_aspect('equal')
    im = ax.imshow(image, cmap=imshow_cmap, interpolation='none', norm=norm,
                   **kwargs)
    im_label = show_label_array(ax, label_array, cmap=cmap, norm=norm,
                                **kwargs)
    return im, im_label